In [3]:
import duckdb as db
import pandas as pd
import numpy as np

import joblib

In [4]:
def sql(query, database = "../data/database.duckdb"):
    con = db.connect(database)
    res = con.execute(query).df()
    con.close()
    return res

In [17]:
sql(
    """ 
    WITH num_votes_quartiles AS
    (SELECT 
    tconst,
    NTILE(4) OVER(ORDER BY numVotes) AS quartile
    FROM title_basics AS tb
    JOIN title_ratings USING(tconst))
    SELECT
    *
    FROM title_basics AS tb
    JOIN num_votes_quartiles USING(tconst)
    JOIN title_ratings USING(tconst)
    WHERE quartile = 4
    ORDER BY numVotes

    """
)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,quartile,averageRating,numVotes
0,tt0027999,movie,Mummy's Boys,Mummy's Boys,0,1936,\N,68,Comedy,4,5.0,316
1,tt0039793,movie,Hidden River,Río Escondido,0,1948,\N,99,Drama,4,7.5,316
2,tt0039873,movie,Suddenly It's Spring,Suddenly It's Spring,0,1947,\N,87,"Comedy,Romance",4,6.3,316
3,tt0040090,movie,The Lovers of Verona,Les amants de Vérone,0,1949,\N,90,"Drama,Romance",4,6.8,316
4,tt0040287,movie,Devil's Cargo,Devil's Cargo,0,1948,\N,64,Mystery,4,5.5,316
...,...,...,...,...,...,...,...,...,...,...,...,...
82392,tt0109830,movie,Forrest Gump,Forrest Gump,0,1994,\N,142,"Drama,Romance",4,8.8,2378710
82393,tt0137523,movie,Fight Club,Fight Club,0,1999,\N,139,"Crime,Drama,Thriller",4,8.8,2464434
82394,tt1375666,movie,Inception,Inception,0,2010,\N,148,"Action,Adventure,Sci-Fi",4,8.8,2682755
82395,tt0468569,movie,The Dark Knight,The Dark Knight,0,2008,\N,152,"Action,Crime,Drama",4,9.0,3020170


In [18]:
sql(
    """ 
    WITH ratings_quartiles AS
    (SELECT 
    tconst,
    NTILE(4) OVER(ORDER BY averageRating) AS rating_quartile
    FROM title_basics AS tb
    JOIN title_ratings USING(tconst))
    
    SELECT
    *
    FROM title_basics AS tb
    JOIN ratings_quartiles USING(tconst)
    JOIN title_ratings USING(tconst)
    WHERE rating_quartile = 4
    ORDER BY averageRating

    """
)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,rating_quartile,averageRating,numVotes
0,tt4602920,movie,Love & Will,Kärlek & vilja,0,2018,\N,85,Drama,4,7.1,51
1,tt4606514,movie,Bobby,Bobby,0,2016,\N,97,"Biography,Documentary,Sport",4,7.1,571
2,tt4607906,movie,Romantic Schemer,Romantic Schemer,0,2015,\N,84,"Comedy,Romance",4,7.1,201
3,tt4613278,movie,Czech Swan,Czech Swan,0,2016,\N,52,Documentary,4,7.1,16
4,tt4620098,movie,Seven Days in Mexico,Seven Days in Mexico,0,2020,\N,70,Documentary,4,7.1,29
...,...,...,...,...,...,...,...,...,...,...,...,...
82392,tt35460760,movie,Parrot Kindergarten,Parrot Kindergarten,0,2025,\N,79,Documentary,4,10.0,17
82393,tt36047815,movie,Through the Pain,Through the Pain,0,2025,\N,\N,Drama,4,10.0,8
82394,tt36609232,movie,Stranger Things: The First Shadow - Broadway,Stranger Things: The First Shadow - Broadway,0,2025,\N,166,Drama,4,10.0,6
82395,tt36671015,movie,Mother Mary,Mother Mary,0,2025,\N,102,\N,4,10.0,633


In [19]:
sql(
    """ 
    WITH ratings_quartiles AS
    (SELECT 
    tconst,
    NTILE(4) OVER(ORDER BY averageRating) AS rating_quartile
    FROM title_basics AS tb
    JOIN title_ratings USING(tconst)),
    num_votes_quartiles AS
    (SELECT 
    tconst,
    NTILE(4) OVER(ORDER BY numVotes) AS votes_quartile
    FROM title_basics AS tb
    JOIN title_ratings USING(tconst))
    
    SELECT
    *
    FROM title_basics AS tb
    JOIN ratings_quartiles USING(tconst)
    JOIN num_votes_quartiles USING(tconst)
    JOIN title_ratings USING(tconst)
    WHERE rating_quartile = 4 and votes_quartile =4
    ORDER BY numVotes 
    """
)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,rating_quartile,votes_quartile,averageRating,numVotes
0,tt27557120,movie,Diane von Furstenberg: Woman in Charge,Diane von Furstenberg: Woman in Charge,0,2024,\N,97,"Biography,Documentary",4,4,7.2,316
1,tt3045760,movie,Kil,Kil,0,2013,\N,90,"Action,Drama,Mystery",4,4,7.2,316
2,tt0039793,movie,Hidden River,Río Escondido,0,1948,\N,99,Drama,4,4,7.5,316
3,tt0102987,movie,Una storia semplice,Una storia semplice,0,1991,\N,91,Drama,4,4,7.2,316
4,tt0155928,movie,Oru Kaidhiyin Diary,Oru Kaidhiyin Diary,0,1985,\N,152,"Action,Crime,Drama",4,4,7.2,316
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15666,tt0109830,movie,Forrest Gump,Forrest Gump,0,1994,\N,142,"Drama,Romance",4,4,8.8,2378710
15667,tt0137523,movie,Fight Club,Fight Club,0,1999,\N,139,"Crime,Drama,Thriller",4,4,8.8,2464434
15668,tt1375666,movie,Inception,Inception,0,2010,\N,148,"Action,Adventure,Sci-Fi",4,4,8.8,2682755
15669,tt0468569,movie,The Dark Knight,The Dark Knight,0,2008,\N,152,"Action,Crime,Drama",4,4,9.0,3020170


In [20]:
sql(
    """ 
    select primaryTitle as Title,
    startYear as Year,
    runtimeMinutes as Runtime,
    nb.primaryName as Director
    from title_basics tb
    join title_akas ta on tconst = titleId
    join title_crew using (tconst)
    join name_basics nb on directors = nconst
    where region like 'US'
"""
)

,Title,Year,Runtime,Director
0,Kryptonite,2015,80,Nicanor Loreti
1,The Birch-Tree Meadow,2003,91,Marceline Loridan Ivens
2,My Brother's Gun,1997,93,Ray Loriga
3,The Age of Love,2014,79,Steven Loring
4,He Hired the Boss,1943,72,Thomas Z. Loring
...,...,...,...,...
307744,Journey to Salem,2023,\N,Alex Avril Klein
307745,A Muse Inn,\N,\N,Blake Davis
307746,Magnolia Flowers,2021,65,Colin Charles Dale
307747,CF Picture's Spider-Man,2018,55,Markos Lee


In [21]:
sql("""
    Select
    primaryTitle,
    primaryName 
    from title_basics
    join title_principals using (tconst)
    join name_basics using (nconst)
    limit 1
    """)

,primaryTitle,primaryName
0,Miss Jerry,Blanche Bayliss


In [22]:
sql("""    
    
    select distinct
    Tconst as tconst,
    tb.primaryTitle as Title,
    tb.genres as Genres,
    startYear as Year,
    runtimeMinutes as Runtime,
    nb.primaryName as Actor,
    nb.primaryName as Director,
    t.overview as Resume,
    tr.averageRating as Rating,
    tr.numVotes as Votes,
    t.Budget as Budget,
    t.Revenue as Revenue,
    production_companies_name as ProductionCompanies,
    production_companies_country as ProductionCountries,
    t.poster_path as Poster
    from title_basics tb
    join title_akas ta on tconst = titleId
    join title_crew using (tconst)
    join name_basics nb on directors = nconst
    join tmdb t on t.imdb_id = tb.tconst
    join title_ratings tr on tb.tconst = tr.tconst
    join title_principals tp on tconst = tp.tconst
    where tp.category like '%actor%' or tp.category like '%actress%'
    ORDER BY Revenue desc
    Limit 1
    """)



,tconst,Title,Genres,Year,Runtime,Actor,Director,Resume,Rating,Votes,Budget,Revenue,ProductionCompanies,ProductionCountries,Poster
0,tt0499549,Avatar,"Action,Adventure,Fantasy",2009,162,James Cameron,James Cameron,"In the 22nd century, a paraplegic Marine is di...",7.9,1430785,237000000,2923706026,"['Dune Entertainment', 'Lightstorm Entertainme...","['US', 'US', 'US', 'GB']",/jRXYjXNq0Cs2TcJjLkki24MLp7u.jpg


In [23]:
df = pd.read_parquet("../data/app_df.gzip")
df.head()

,tconst,Title,Genres,Year,Runtime,Actors,Director,Writers,Resume,Rating,Votes,Budget,Revenue,Profit,ProductionCompanies,ProductionCountries,Poster
0,tt0499549,Avatar,"Action,Adventure,Fantasy",2009,162,"Zoe Saldaña, Giovanni Ribisi, CCH Pounder, Mic...",James Cameron,James Cameron,"In the 22nd century, a paraplegic Marine is di...",7.9,1430785,237000000,2923706026,2686706026,"['Dune Entertainment', 'Lightstorm Entertainme...","['US', 'US', 'US', 'GB']",/jRXYjXNq0Cs2TcJjLkki24MLp7u.jpg
1,tt4154796,Avengers: Endgame,"Action,Adventure,Sci-Fi",2019,181,"Jeremy Renner, Don Cheadle, Chris Evans, Chadw...","Anthony Russo, Joe Russo","Christopher Markus, Steve Englehart, Jack Kirb...",After the devastating events of Avengers: Infi...,8.4,1357117,356000000,2794731755,2438731755,['Marvel Studios'],None,/or06FN3Dka5tukK1e9sl16pB3iy.jpg
2,tt0120338,Titanic,"Drama,Romance",1997,194,"Kate Winslet, Frances Fisher, Billy Zane, Davi...",James Cameron,James Cameron,101-year-old Rose DeWitt Bukater tells the sto...,7.9,1339359,200000000,2264162353,2064162353,"['Paramount', '20th Century Fox', 'Lightstorm ...","['US', 'US', 'US']",/9xjZS2rlVxm8SFx8kPC3aIGCOYQ.jpg
3,tt1630029,Avatar: The Way of Water,"Action,Adventure,Fantasy",2022,192,"Cliff Curtis, CCH Pounder, Sam Worthington, Si...",James Cameron,"Rick Jaffa, Shane Salerno, James Cameron, Aman...",Set more than a decade after the events of the...,7.5,534023,460000000,2319331580,1859331580,"['20th Century Studios', 'Lightstorm Entertain...",None,/t6HIqrRAclMCA60NsSmeqe9RmNV.jpg
4,tt2488496,Star Wars: Episode VII - The Force Awakens,"Action,Adventure,Sci-Fi",2015,138,"Domhnall Gleeson, Lupita Nyong'o, Mark Hamill,...",J.J. Abrams,"George Lucas, Michael Arndt, J.J. Abrams, Lawr...",Thirty years after defeating the Galactic Empi...,7.8,999627,245000000,2068223624,1823223624,"['Lucasfilm Ltd.', 'Bad Robot']",None,/wqnLdwVXoBjKibFRR5U3y0aDUhs.jpg


In [24]:
genre_series = df['Genres'].str.split(',')
genre_list = [genre for sublist in genre_series for genre in sublist]
genre_list = list(filter(lambda x: x not in ['\\N', 'Adult'], genre_list))
genre_list = pd.Series(genre_list)

In [25]:
genre_list.unique()

array(['Action', 'Adventure', 'Fantasy', 'Sci-Fi', 'Drama', 'Romance',
       'Animation', 'Thriller', 'Comedy', 'Family', 'Crime', 'Biography',
       'Music', 'History', 'Horror', 'Mystery', 'Musical', 'War',
       'Western', 'Sport', 'Documentary', 'Film-Noir', 'Reality-TV',
       'News'], dtype=object)

In [26]:
df[df['Poster'].isnull()]

,tconst,Title,Genres,Year,Runtime,Actors,Director,Writers,Resume,Rating,Votes,Budget,Revenue,Profit,ProductionCompanies,ProductionCountries,Poster
1238,tt3718222,Someone Wants to Talk with You,"Drama,Family",2012,90,"Anahita Ne'mati, Sara Esmael, Shahin Taslimi, ...",Manouchehr Hadi,"Saeed Dolatkhani, Manouchehr Hadi",Yasaman has been living with her mother Leila ...,5.5,305,100000,100000000,99900000,['Iranian Independents'],['IR'],None
4991,tt3180664,Yajamana,"Drama,Family",2000,162,"Shashikumar, Ramesh Bhat, Avinash, M.N. Lakshm...","Sheshadri, Radha Bharathi","Radha Bharathi, Vikraman, Ravi Srivatsa, Shesh...",Shankar sacrifices his own interests and refus...,8.3,481,200000,8000000,7800000,[],[],None
5580,tt5465374,Shivalinga,"Drama,Horror,Thriller",2016,155,"Malavika Avinash, Ashok, Urvashi, Vedhika, Avi...",P. Vasu,P. Vasu,Shivalinga is story weaved around a murder mys...,6.9,675,0,5500000,5500000,[],[],None
7609,tt2391817,Ee Rojullo,"Comedy,Romance",2012,129,"M.S. Narayana, Kumar Sai Pampana, Madhumani, R...",Maruthi Dasari,Maruthi Dasari,Srinivas is a gullible lover who gets used by ...,5.5,165,610000,1800000,1190000,['Good Cinema Group'],None,None
7806,tt2100411,Bodyguard,"Action,Comedy",2012,150,"Dharmavarapu Subramanyam, Saloni Aswani, Trish...",Gopichand Malineni,"Gopichand Malineni, Siddique, Kona Venkat","A remake tale, the story begins with Venkatadr...",5.1,522,8000000,9000000,1000000,[],None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97988,tt1537302,Angel Camouflaged,"Drama,Music",2010,116,"Patty Smyth, Violeta Leskyte Cucchiara, Warric...",R. Michael Givens,"R. Michael Givens, Frederick Bailey, Ken Dalton","This rock and roll drama stars, James Brolin, ...",5.2,145,4000000,0,-4000000,[],None,None
98256,tt0214277,Very Mean Men,"Comedy,Crime",2000,93,"Louise Fletcher, Martin Landau, Billy Drago, M...",Tony Vitale,Paul T. Murray,A bartender wants rid of an obnoxious drunk bu...,5.9,308,4500000,0,-4500000,"['Miracle Entertainment', 'Baio / White Produc...",None,None
98857,tt0079504,The Magician of Lublin,Drama,1979,105,"Shelley Winters, Maia Danziger, Lou Jacobi, Va...",Menahem Golan,"Irving S. White, Joseph Gross, Barbara Dana, M...",Yasha is a Jewish stage magician who tours thr...,5.8,190,6000000,0,-6000000,"['NF Geria III-Produktion München', 'Geria III...",None,None
99831,tt1961680,"Young, High and Dead","Horror,Thriller",2013,89,"Hannah Tointon, Su Elliot, Lara Wollington, Ni...","Katie Anna Brady, Daniel Fenton, Luke Brady, T...",Luke Brady,When five friends head to the woods for a week...,3.0,157,10000000,0,-10000000,"['Salani Films', 'Rum Punch Productions']",None,None


In [5]:
#Conversion de notre base de donnée acquise via SQL en DataFrame
data = pd.DataFrame(sql("""
    SELECT
        *
    FROM
       dataframe_view
    """))

In [23]:
# Make a real copy
data_copy = data.copy()

# ... all your cleaning steps ...

# Reset the index so it matches the row order
data_copy.reset_index(drop=True, inplace=True)


In [24]:
def clean_row(row):
    return row.replace("[", "").replace("]", "").replace("'", "")
data_copy.drop(columns= ['Profit','Revenue', 'Budget'], inplace=True) #trop de valeurs manquantes, DF faussé
data_copy['ProductionCountries'] = data_copy['ProductionCountries'].apply(lambda x: 'Unknown' if x =='\\N' else x)
data_copy['Writers'] = data_copy['Writers'].apply(lambda x: 'Unknown' if x== '\\N'  else x)
data_copy['Resume'] = data_copy['Resume'].apply(lambda x: 'Unknown' if x == '\\N'  else x)
data_copy = data_copy.dropna(subset=['Poster'])
#nettoyage de la colonne Year (il semblerait qu'il y ait qu'une ligne avec une valeur '\N')
data_copy['Year'] = data_copy['Year'].replace('\\N', np.nan)
data_copy = data_copy.dropna(subset=['Year'])
#On convertit la colonne Year en int pour pouvoir l'utiliser dans le modèle
data_copy['Year'] = data_copy['Year'].astype(int)
data_copy['Resume'].fillna('Unknown', inplace=True)
df_runtime = data_copy[data_copy['Runtime'] != '\\N']
df_runtime['Runtime'] = df_runtime['Runtime'].astype(int)
data_copy['Runtime'] = data_copy['Runtime'].apply(lambda x: df_runtime['Runtime'].median() if x == '\\N' else x)
data_copy['Runtime'] = data_copy['Runtime'].astype(int)
data_copy['Runtime'].describe()
data_copy['ProductionCountries'].fillna('Unknown', inplace=True)
data_copy['ProductionCountries'] = data_copy['ProductionCountries'].apply(clean_row)
data_copy['ProductionCompanies'] = data_copy['ProductionCompanies'].apply(clean_row)
data_copy['ProductionCountries'] = data_copy['ProductionCountries'].apply(
    lambda x: ', '.join(dict.fromkeys(str(x).split(', ')))
)
data_copy['Writers'].fillna('Unknown', inplace=True)

C:\Users\jeanp\AppData\Local\Temp\ipykernel_11456\272718629.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_copy['Resume'].fillna('Unknown', inplace=True)
C:\Users\jeanp\AppData\Local\Temp\ipykernel_11456\272718629.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_runtime

In [25]:
#import de la bibliothèque scikit-learn
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer  
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
import numpy as np
from sklearn.preprocessing import FunctionTransformer


In [26]:
num_cols = data_copy.select_dtypes('number')
non_num_cols = data_copy.select_dtypes(exclude='number')
non_num_cols = non_num_cols.drop(columns=['tconst', 'Poster', 'ProductionCountries'])
non_num_cols = non_num_cols.select_dtypes(include='object').columns.tolist()
num_cols = data_copy.select_dtypes(include=['int64', 'float64']).columns.to_list()

In [27]:
num_cols, non_num_cols

(['Year', 'Runtime', 'Rating', 'Votes'],
 ['Title',
  'Genres',
  'Actors',
  'Director',
  'Writers',
  'Resume',
  'ProductionCompanies'])

In [28]:
# On crée une liste de tuples pour appliquer un TfidfVectorizer à chaque colonne texte
text_transformers = [
    (col, TfidfVectorizer(stop_words='english'), col)
    for col in non_num_cols
]
log_transformer = FunctionTransformer(lambda x: np.log1p(x))

all_transformers = text_transformers +[
    ('num', Pipeline([
        ('log_votes', ColumnTransformer([
            ('log', log_transformer, ['Votes']),
            ('pass', 'passthrough', [col for col in num_cols if col != 'Votes'])
        ])),
        ('scale', StandardScaler())
    ]), num_cols)
]

preprocessor = ColumnTransformer(transformers=all_transformers)

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('knn', NearestNeighbors(n_neighbors=6, metric='cosine'))
])
X = data_copy[num_cols + non_num_cols]
film_index = 131
pipeline.fit(X)
vectorized_data = pipeline.named_steps['preprocessor'].transform(X)
distances, indices = pipeline.named_steps['knn'].kneighbors(vectorized_data.getrow(film_index))
print("Distances:", distances)
print("Indices:", indices)
for i in indices[0]:
    print(data_copy['Title'].iloc[i])
    print(data_copy['Resume'].iloc[i])
    print(data_copy['Director'].iloc[i])
    print()

Distances: [[0.         0.13870515 0.15580586 0.15649186 0.1669232  0.1683212 ]]
Indices: [[  131   182    47    86   287 98108]]
Up
Carl Fredricksen spent his entire life dreaming of exploring the globe and experiencing life to its fullest. But at age 78, life seems to have passed him by, until a twist of fate (and a persistent 8-year old Wilderness Explorer named Russell) gives him a new lease on life.
Bob Peterson, Pete Docter

Monsters, Inc.
James Sullivan and Mike Wazowski are monsters, they earn their living scaring children and are the best in the business... even though they're more afraid of the children than they are of them. When a child accidentally enters their world, James and Mike suddenly find that kids are not to be afraid of and they uncover a conspiracy that could threaten all children across the world.
Lee Unkrich, David Silverman, Pete Docter

Finding Nemo
Nemo, an adventurous young clownfish, is unexpectedly taken from his Great Barrier Reef home to a dentist's of

In [13]:
distances, indices = pipeline.named_steps['knn'].kneighbors(vectorized_data.getrow(217))
print("Distances:", distances)
print("Indices:", indices)
for i in indices[0]:
    print(data_copy['Title'].iloc[i])
    print(data_copy['Resume'].iloc[i])
    print(data_copy['Director'].iloc[i])
    print()

Distances: [[1.11022302e-16 2.48294075e-01 2.50233786e-01 2.64268621e-01
  2.66153520e-01 2.73962971e-01]]
Indices: [[ 217   45  187   64  289 2489]]
Mrs. Doubtfire
Loving but irresponsible dad Daniel Hillard, estranged from his exasperated spouse, is crushed by a court order allowing only weekly visits with his kids. When Daniel learns his ex needs a housekeeper, he gets the job -- disguised as a British nanny. Soon he becomes not only his children's best pal but the kind of parent he should have been from the start.
Chris Columbus

Harry Potter and the Sorcerer's Stone
Harry Potter has lived under the stairs at his aunt and uncle's house his whole life. But on his 11th birthday, he learns he's a powerful wizard—with a place waiting for him at the Hogwarts School of Witchcraft and Wizardry. As he learns to harness his newfound powers with the help of the school's kindly headmaster, Harry uncovers the truth about his parents' deaths—and about the villain who's to blame.
Chris Columbus


In [29]:
_, indices = pipeline.named_steps['knn'].kneighbors(vectorized_data)


In [13]:
data_copy.shape

(99312, 14)

In [36]:
joblib.dump(pipeline, '../KNN/model.pkl')

['../KNN/model.pkl']

In [37]:
data_copy.to_parquet('../data/st_df.gzip', compression='gzip')

In [17]:
data_copy.describe().T

,count,mean,std,min,25%,50%,75%,max
Year,99312.0,1998.527328,21.535719,1941.0,1985.0,2006.0,2016.0,2025.0
Runtime,99312.0,100.504461,20.071235,26.0,88.0,96.0,108.0,354.0
Rating,99312.0,5.819317,1.234566,1.0,5.1,6.0,6.7,9.5
Votes,99312.0,11365.278949,66024.575229,101.0,248.0,630.0,2389.0,3043168.0


In [30]:
data_copy['similar_films'] = list(indices)

In [31]:
data_copy['similar_films'] = data_copy['similar_films'].apply(lambda x : x[1:])

In [47]:
data_copy[data_copy['Title'] == 'Mrs. Doubtfire']

,tconst,Title,Genres,Year,Runtime,Actors,Director,Writers,Resume,Rating,Votes,ProductionCompanies,ProductionCountries,Poster,similar_films
217,tt0107614,Mrs. Doubtfire,"Comedy, Drama, Family",1993,125,"Robin Williams, Pierce Brosnan, Polly Holliday...",Chris Columbus,"Leslie Dixon, Randi Mayem Singer, Anne Fine","Loving but irresponsible dad Daniel Hillard, e...",7.1,304531,"20th Century Fox, Blue Wolf Productions","US,",/shHrSmXS5140o6sQzgzXxn3KqSm.jpg,"[45, 187, 64, 289, 2489]"


In [50]:
data_copy[data_copy['Title'] == 'Fight Club']

,tconst,Title,Genres,Year,Runtime,Actors,Director,Writers,Resume,Rating,Votes,ProductionCompanies,ProductionCountries,Poster,similar_films
2490,tt0137523,Fight Club,"Crime, Drama, Thriller",1999,139,"David Andrews, George Maguire, Zach Grenier, E...",David Fincher,"Jim Uhls, Chuck Palahniuk",A ticking-time-bomb insomniac and a slippery s...,8.8,2464434,"Regency Enterprises, Fox 2000 Pictures, Taurus...","US, DE,",/pB8BM7pdSp6B6Ih7QZ4DrQ3PmJK.jpg,"[357, 328, 581, 54, 602]"


In [32]:
data_copy.to_parquet('../data/movie_df.gzip', compression='gzip')